In [1]:
import os
import pandas as pd
import pickle
import json


from src.dataset.dataset_info import datasets, DatasetInfo
from src.dataset.clean_dataset import clean_dataset
from src.dataset.create_class_num_col import one_dataset_class_num_col
from src.dataset.features_analysis import feature_analysis_pipeline
from src.dataset.undersample_classes import undersample_classes
from src.graph.graph_measures import calculate_graph_measures
from local_variables import datasets_main_path

In [2]:
with_sort_timestamp = False
with_undersample_classes = False

# name = "cic_ids_2017_5_percent"
# original_path = "./testing_dfs/cic_ids_2017_5_percent.parquet"
# file_type = "parquet"

# name = "cic_ton_iot_5_percent"
# original_path = "./testing_dfs/cic_ton_iot_5_percent.parquet"
# file_type = "parquet"

# name = "cic_ton_iot"
# original_path = datasets_main_path + "cic_ton_iot.parquet"
# file_type = "parquet"

# name = "cic_ids_2017"
# original_path = datasets_main_path + "cic_ids_2017.parquet"
# file_type = "parquet"

# name = "cic_bot_iot"
# original_path = datasets_main_path + "cic_bot_iot.parquet"
# file_type = "parquet"

# name = "cic_ton_iot_modified"
# original_path = datasets_main_path + "CIC-ToN-IoT-Modified.pkl"
# file_type = "pkl"

# name = "nf_ton_iotv2_modified"
# original_path = datasets_main_path + "NF-ToN-IoT-v2-Modified.pkl"
# file_type = "pkl"

# name = "ccd_inid_modified"
# original_path = datasets_main_path + "CCD-INID-Modified.pkl"
# file_type = "pkl"

# name = "nf_uq_nids_modified"
# original_path = datasets_main_path + "NF-UQ-NIDS-Modified.pkl"
# file_type = "pkl"

# name = "edge_iiot"
# original_path = datasets_main_path + "edge_iiot.pkl"
# file_type = "pkl"

# name = "nf_cse_cic_ids2018"
# original_path = datasets_main_path + "NF-CSE-CIC-IDS2018.csv"
# file_type = "csv"

# name = "nf_bot_iotv2"
# original_path = datasets_main_path + "NF-BoT-IoT-v2.csv"
# file_type = "csv"

# name = "nf_uq_nids"
# original_path = datasets_main_path + "NF-UQ-NIDS.csv"
# file_type = "csv"

name = "x_iiot"
original_path = datasets_main_path + "X-IIoTID dataset.csv"
file_type = "csv"

folder_path = os.path.join("datasets", name)
output_path = os.path.join(folder_path, name + ".parquet")

if not os.path.isdir(folder_path):
    os.mkdir(folder_path)

print(f"==>> original_path: {original_path}")
print(f"==>> folder_path: {folder_path}")
print(f"==>> output_path: {output_path}")

==>> original_path: c:\Users\Administrateur\Desktop\datasets\X-IIoTID dataset.csv
==>> folder_path: datasets\x_iiot
==>> output_path: datasets\x_iiot\x_iiot.parquet


# Preparing Datasets

### Reading and Cleaning

In [3]:
if file_type == "parquet":
    df = pd.read_parquet(original_path)
if file_type == "csv":
    df = pd.read_csv(original_path)
if file_type == "pkl":
    df = pd.read_pickle(original_path)
    # df = pd.read_pickle(original_path, compression="zip")

C:\Users\Administrateur\AppData\Local\Temp\1\ipykernel_6556\3654495911.py:4: DtypeWarning: Columns (1,3,5,12,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(original_path)


In [4]:
pd.options.display.max_columns = df.shape[1]


In [5]:
df.head()


,Date,Timestamp,Scr_IP,Scr_port,Des_IP,Des_port,Protocol,Service,Duration,Scr_bytes,Des_bytes,Conn_state,missed_bytes,is_syn_only,Is_SYN_ACK,is_pure_ack,is_with_payload,FIN or RST,Bad_checksum,is_SYN_with_RST,Scr_pkts,Scr_ip_bytes,Des_pkts,Des_ip_bytes,anomaly_alert,total_bytes,total_packet,paket_rate,byte_rate,Scr_packts_ratio,Des_pkts_ratio,Scr_bytes_ratio,Des_bytes_ratio,Avg_user_time,Std_user_time,Avg_nice_time,Std_nice_time,Avg_system_time,Std_system_time,Avg_iowait_time,Std_iowait_time,Avg_ideal_time,Std_ideal_time,Avg_tps,Std_tps,Avg_rtps,Std_rtps,Avg_wtps,Std_wtps,Avg_ldavg_1,Std_ldavg_1,Avg_kbmemused,Std_kbmemused,Avg_num_Proc/s,Std_num_proc/s,Avg_num_cswch/s,std_num_cswch/s,OSSEC_alert,OSSEC_alert_level,Login_attempt,Succesful_login,File_activity,Process_activity,read_write_physical.process,is_privileged,class1,class2,class3
0,9/01/2020,1578540956,192.168.2.199,49278,192.168.2.10,80,tcp,http,0.67369,13437,34924,1,0,True,True,True,True,True,False,False,105,18905,105,40392,TRUE,107658,210,311.7160712,159803.4704,0.5,0.5,0.300414275,0.699585725,9.207,5.55584206,10.994,1.356305275,4.864,1.873004004,0.311,0.224653066,74.624,8.245611196,12.297,10.38585004,8,10.50714043,4.297,2.723578712,2.146,0.102781321,915852.8,2507.97563,5.1,3.238826948,2806.2,158.7493622,1,5,0,0,0,0,0,0,Scanning_vulnerability,Reconnaissance,Attack
1,13/01/2020,1578871873,10.0.1.5,39769,131.236.3.92,53,udp,dns,0.000083,78,0,1,0,False,False,False,True,False,False,False,2,134,0,0,FALSE,212,2,24096.38554,2554216.867,1,0,1,0,12.326,6.674247823,2.895,1.604302029,3.778,2.099741889,2.741,5.327471164,78.263,8.869399134,6.6,5.713142743,0,0,6.6,5.713142743,0.612,0.037629775,910498.4,4044.112046,1,0.894427191,1561.6,247.0802299,0,0,0,0,0,0,0,0,Normal,Normal,Normal
2,9/01/2020,1578522486,172.24.1.80,59050,172.24.1.1,53,udp,dns,0.000132,38,38,1,0,False,False,False,True,False,False,False,1,66,1,66,FALSE,208,2,15151.51515,1575757.576,0.5,0.5,0.5,0.5,6.931,6.416007248,0.706,0.408905857,1.693,0.771635277,2.423,3.829809525,88.245,7.112108337,37.4,40.19004852,30.1,39.79811553,7.3,3.1,0.55,0.02,921020.4,2139.652645,1,0,1603.3,294.1390997,0,0,0,0,0,0,0,0,Normal,Normal,Normal
3,27/02/2020,1582757640,192.168.2.196,37966,192.168.2.10,1880,tcp,websocket,9.378481,1121,484,1,0,True,True,True,True,True,False,False,8,1545,6,804,FALSE,3954,14,1.492779055,421.6034558,0.571428571,0.428571429,0.67425392,0.32574608,10.244,7.932040343,1.965,1.514168088,3.941,1.709055002,0.509,0.425522032,83.339,10.20608686,29,26.57442379,23.3,26.12680616,5.7,6.229767251,0.708,0.026,915874.8,977.8851466,7.2,5.325410782,1685.3,467.8737116,0,0,1,1,1,1,1,1,Normal,Normal,Normal
4,16/12/2019,1576452612,172.24.1.80,38233,172.24.1.1,53,udp,dns,0.000074,-,-,1,0,False,False,False,True,False,False,False,-,-,-,-,FALSE,-,-,-,-,-,-,-,-,5.181,5.980458929,2.365,1.284462923,2.415,1.401151312,1.803,4.478256469,88.239,8.090017862,9.8,1.326649916,0.1,0.3,9.7,1.268857754,0.536,0.030066593,918213.2,2342.93836,0.6,0.489897949,1456.5,346.8847791,0,0,0,0,0,0,0,0,Normal,Normal,Normal


In [6]:
timestamp_format = "mixed"
# timestamp_format="%d/%m/%Y %I:%M:%S %p"


In [7]:
df.columns


Index(['Date', 'Timestamp', 'Scr_IP', 'Scr_port', 'Des_IP', 'Des_port',
       'Protocol', 'Service', 'Duration', 'Scr_bytes', 'Des_bytes',
       'Conn_state', 'missed_bytes', 'is_syn_only', 'Is_SYN_ACK',
       'is_pure_ack', 'is_with_payload', 'FIN or RST', 'Bad_checksum',
       'is_SYN_with_RST', 'Scr_pkts', 'Scr_ip_bytes', 'Des_pkts',
       'Des_ip_bytes', 'anomaly_alert', 'total_bytes', 'total_packet',
       'paket_rate', 'byte_rate', 'Scr_packts_ratio', 'Des_pkts_ratio',
       'Scr_bytes_ratio', 'Des_bytes_ratio', 'Avg_user_time', 'Std_user_time',
       'Avg_nice_time', 'Std_nice_time', 'Avg_system_time', 'Std_system_time',
       'Avg_iowait_time', 'Std_iowait_time', 'Avg_ideal_time',
       'Std_ideal_time', 'Avg_tps', 'Std_tps', 'Avg_rtps', 'Std_rtps',
       'Avg_wtps', 'Std_wtps', 'Avg_ldavg_1', 'Std_ldavg_1', 'Avg_kbmemused',
       'Std_kbmemused', 'Avg_num_Proc/s', 'Std_num_proc/s', 'Avg_num_cswch/s',
       'std_num_cswch/s', 'OSSEC_alert', 'OSSEC_alert_level', 'Lo

In [8]:
list(df.dtypes)

[dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('int64'),
 dtype('O'),
 dtype('bool'),
 dtype('bool'),
 dtype('bool'),
 dtype('bool'),
 dtype('bool'),
 dtype('bool'),
 dtype('bool'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('O'),
 dtype('O'),
 dtype('O')]

In [9]:
df.describe(include="all")

,Date,Timestamp,Scr_IP,Scr_port,Des_IP,Des_port,Protocol,Service,Duration,Scr_bytes,Des_bytes,Conn_state,missed_bytes,is_syn_only,Is_SYN_ACK,is_pure_ack,is_with_payload,FIN or RST,Bad_checksum,is_SYN_with_RST,Scr_pkts,Scr_ip_bytes,Des_pkts,Des_ip_bytes,anomaly_alert,total_bytes,total_packet,paket_rate,byte_rate,Scr_packts_ratio,Des_pkts_ratio,Scr_bytes_ratio,Des_bytes_ratio,Avg_user_time,Std_user_time,Avg_nice_time,Std_nice_time,Avg_system_time,Std_system_time,Avg_iowait_time,Std_iowait_time,Avg_ideal_time,Std_ideal_time,Avg_tps,Std_tps,Avg_rtps,Std_rtps,Avg_wtps,Std_wtps,Avg_ldavg_1,Std_ldavg_1,Avg_kbmemused,Std_kbmemused,Avg_num_Proc/s,Std_num_proc/s,Avg_num_cswch/s,std_num_cswch/s,OSSEC_alert,OSSEC_alert_level,Login_attempt,Succesful_login,File_activity,Process_activity,read_write_physical.process,is_privileged,class1,class2,class3
count,820503,820537,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834.000000,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834.000000,820834.000000,820834.000000,820834.000000,820834.000000,820834.000000,820834.000000,820834.000000,820834,820834,820834
unique,49,273175,36,94481,96,10610,4,17,330207,21054,51899,NaN,249,2,2,2,2,2,1,1,1211,32872,1556,55574,5,62464,2216,303353,339658,19995,19995,72716,72716,12635,51646,4551,50541,5174,50456,10157,30703,19705,51725,3736,21771,2254,16492,2017,11124,2814,3450,28144,51523,1061,3649,14986,51588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,10,2
top,24/02/2020,1578542789,192.168.2.199,5353,192.168.2.10,80,tcp,http,-,-,-,NaN,0,True,False,False,True,True,False,False,1,-,1,0,FALSE,-,2,-,-,0.5,0.5,-,-,6.478,6.07937176,1.472,1.403424383,2.15,2.563215949,0.077,0.117647779,87.054,7.736539537,3,0,0,0,3,0,0.825,0.02,914762.8,2614.998616,0,0,1578.8,393.7089788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,Normal,Normal
freq,147621,4387,156023,2247,312684,184935,422334,237738,75761,223871,223871,NaN,638581,420448,497805,497546,711816,415483,820834,820834,249056,152743,208246,174182,576122,223872,260010,223871,223871,248977,248977,223872,223872,5776,5707,6071,5707,6365,5707,12923,13186,5754,5707,11726,11484,138135,138134,33190,32601,6766,37463,5755,5707,131979,147442,5804,5707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,421417,421417,421417
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.851623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.051186,0.267717,0.087305,0.082730,0.072650,0.082620,0.355309,0.082456,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.355474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.220377,1.178027,0.282282,0.275475,0.259562,0.275307,0.478607,0.275059,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [10]:
datesetInfo = DatasetInfo(name="x_iiot",
                          path="./datasets/x_iiot/x_iiot.parquet",
                          file_type="parquet",
                          src_ip_col="Scr_IP",
                          src_port_col="Scr_port",
                          dst_ip_col="Des_IP",
                          dst_port_col="Des_port",
                          flow_id_col=None,
                          timestamp_col="Timestamp",
                          label_col="class3",
                          class_col="class2",
                          class_num_col="Class",
                          timestamp_format="mixed",
                          drop_columns=["Scr_IP", "Scr_port", "Des_IP",
                                        "Des_port", "Timestamp", "class1", "class2"],
                          )

In [11]:
df.shape


(820834, 68)

In [12]:
# import numpy as np
# df.replace([np.inf, -np.inf], np.nan, inplace=True)


In [13]:
df.describe()


,Conn_state,OSSEC_alert,OSSEC_alert_level,Login_attempt,Succesful_login,File_activity,Process_activity,read_write_physical.process,is_privileged
count,820834.000000,820834.000000,820834.000000,820834.000000,820834.000000,820834.000000,820834.000000,820834.000000,820834.000000
mean,0.851623,0.051186,0.267717,0.087305,0.082730,0.072650,0.082620,0.355309,0.082456
std,0.355474,0.220377,1.178027,0.282282,0.275475,0.259562,0.275307,0.478607,0.275059
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
df.isna().sum()


Date                           331
Timestamp                      297
Scr_IP                           0
Scr_port                         0
Des_IP                           0
                              ... 
read_write_physical.process      0
is_privileged                    0
class1                           0
class2                           0
class3                           0
Length: 68, dtype: int64

In [15]:
def to_categorical_codes(column):
    return pd.Categorical(column).codes


if name == 'x_iiot':
    # columns_to_convert = [
    #     'Protocol', 'Service', 'is_syn_only', 'Is_SYN_ACK',
    #     'is_pure_ack', 'is_with_payload', 'FIN or RST',
    #     'Bad_checksum', 'is_SYN_with_RST', 'anomaly_alert']

    # for col in columns_to_convert:
    #     if col in df.columns:
    #         df[col] = to_categorical_codes(df[col])
    #     else:
    #         print(f"Warning: Column '{col}' not found in the DataFrame")

    object_columns_to_keep = ['Timestamp', 'Scr_IP', 'Scr_port',
                              'Des_IP', 'Des_port', 'class1', 'class2', 'class3']

    df.drop(['Date'], axis=1, inplace=True)
    # df['Timestamp'] = df['Timestamp'].apply(str)

    df['Timestamp'] = pd.to_datetime(
            df['Timestamp'].str.strip(), unit='s', errors='coerce')
    df.dropna(subset=['Timestamp'], inplace=True)
    
    # Convert all other columns to int
    for col in df.columns:
        if df[col].dtype == 'object' and col not in object_columns_to_keep:
            df[col] = to_categorical_codes(df[col])

    df['class3'] = (df['class3'] == 'Attack').astype(int)
    # df.rename(columns={'class3': 'label'}, inplace=True)

    df = df.applymap(lambda x: 0 if x == '-' else x)
    df = df.applymap(lambda x: 0 if x == '?' else x)
    df = df.applymap(lambda x: 0 if x == '#DIV/0!' else x)
    df = df.applymap(lambda x: 0 if x == 'excel' else x)
    df = df.applymap(lambda x: 0 if x == 'aza' else x)
    df = df.applymap(lambda x: 0 if x == ' ' else x)

if name == 'edge_iiot':
    df.drop(['tcp.options', 'tcp.payload'], axis=1, inplace=True)

    columns_to_convert = ['mqtt.protoname', 'mqtt.topic', 'mqtt.conack.flags', 'mqtt.msg', 'http.request.method', 'dns.qry.name.len', 'arp.src.proto_ipv4',
                          'http.request.full_uri', 'http.file_data', 'http.request.version', 'arp.dst.proto_ipv4', 'http.request.uri.query', 'tcp.srcport', 'http.referer']

    for col in columns_to_convert:
        if col in df.columns:
            df[col] = to_categorical_codes(df[col])
        else:
            print(f"Warning: Column '{col}' not found in the DataFrame")


if name == 'ccd_inid_modified':
    columns_to_convert = ['splt_direction', 'splt_ps', 'splt_piat_ms', 'application_name',
                          'application_category_name', 'requested_server_name', 'client_fingerprint']

    for col in columns_to_convert:
        if col in df.columns:
            df[col] = to_categorical_codes(df[col])
        else:
            print(f"Warning: Column '{col}' not found in the DataFrame")
    df['traffic_type'] = (df['traffic_type'] == 'attack').astype(int)
    df.drop(['requested_server_name', 'client_fingerprint',
            'server_fingerprint', 'user_agent', 'content_type'], axis=1, inplace=True)

if name == 'cic_ton_iot_modified':
    df.drop(['datetime'], axis=1, inplace=True)

C:\Users\Administrateur\AppData\Local\Temp\1\ipykernel_6556\3539214660.py:23: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['Timestamp'] = pd.to_datetime(
C:\Users\Administrateur\AppData\Local\Temp\1\ipykernel_6556\3539214660.py:35: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: 0 if x == '-' else x)
C:\Users\Administrateur\AppData\Local\Temp\1\ipykernel_6556\3539214660.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: 0 if x == '?' else x)
C:\Users\Administrateur\AppData\Local\Temp\1\ipykernel_6556\3539214660.py:37: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  d

In [16]:
df = clean_dataset(df, flow_id_col=datesetInfo.flow_id_col,
                   timestamp_col=datesetInfo.timestamp_col)

==>> original df.shape[0]: 638604
==>> after drop na df.shape[0]: 638604
==>> after drop_duplicates df.shape[0]: 631814


In [17]:
df[datesetInfo.src_ip_col] = df[datesetInfo.src_ip_col].apply(str)
if datesetInfo.src_port_col:

    df[datesetInfo.src_port_col] = df[datesetInfo.src_port_col].apply(str)

df[datesetInfo.dst_ip_col] = df[datesetInfo.dst_ip_col].apply(str)
if datesetInfo.dst_port_col:

    df[datesetInfo.dst_port_col] = df[datesetInfo.dst_port_col].apply(str)

In [18]:
df.head()


,Timestamp,Scr_IP,Scr_port,Des_IP,Des_port,Protocol,Service,Duration,Scr_bytes,Des_bytes,Conn_state,missed_bytes,is_syn_only,Is_SYN_ACK,is_pure_ack,is_with_payload,FIN or RST,Bad_checksum,is_SYN_with_RST,Scr_pkts,Scr_ip_bytes,Des_pkts,Des_ip_bytes,anomaly_alert,total_bytes,total_packet,paket_rate,byte_rate,Scr_packts_ratio,Des_pkts_ratio,Scr_bytes_ratio,Des_bytes_ratio,Avg_user_time,Std_user_time,Avg_nice_time,Std_nice_time,Avg_system_time,Std_system_time,Avg_iowait_time,Std_iowait_time,Avg_ideal_time,Std_ideal_time,Avg_tps,Std_tps,Avg_rtps,Std_rtps,Avg_wtps,Std_wtps,Avg_ldavg_1,Std_ldavg_1,Avg_kbmemused,Std_kbmemused,Avg_num_Proc/s,Std_num_proc/s,Avg_num_cswch/s,std_num_cswch/s,OSSEC_alert,OSSEC_alert_level,Login_attempt,Succesful_login,File_activity,Process_activity,read_write_physical.process,is_privileged,class1,class2,class3
0,2020-01-09 03:35:56,192.168.2.199,49278,192.168.2.10,80,1,4,75756,1849,13843,1,0,True,True,True,True,True,False,False,25,7529,57,17208,1,917,913,167694,43725,16027,1159,44278,13909,11423,27335,1455,27902,4139,42866,113,1057,2193,40997,333,734,1640,494,1026,5953,2047,1005,16196,27966,574,1178,13398,3974,1,5,0,0,0,0,0,0,Scanning_vulnerability,Reconnaissance,1
1,2020-01-12 23:31:13,10.0.1.5,39769,131.236.3.92,53,2,2,83,16696,1,1,0,False,False,False,True,False,False,False,296,3276,1,1,0,29023,868,148000,100503,17185,1,58186,1,2744,37376,2400,35644,3253,45533,4108,23469,3505,44749,2892,17266,1,1,1206,7535,578,190,6190,41653,78,238,8311,16722,0,0,0,0,0,0,0,0,Normal,Normal,0
2,2020-01-08 22:28:06,172.24.1.80,59050,172.24.1.1,53,2,2,132,10806,14586,1,0,False,False,False,True,False,False,False,2,25357,2,32292,0,28664,868,98902,41993,16027,1159,53534,4653,9190,35462,223,1981,660,6866,3804,21387,11589,33713,2438,16476,1126,12046,1269,6490,513,62,25823,21218,78,2,8932,25165,0,0,0,0,0,0,0,0,Normal,Normal,0
3,2020-02-26 22:54:00,192.168.2.196,37966,192.168.2.10,1880,1,16,260329,689,16960,1,0,True,True,True,True,True,False,False,1020,4938,1013,39562,0,38772,443,62894,148816,16906,280,56990,1197,1256,44269,1406,33028,3414,39860,249,4971,6960,2339,2040,12882,926,9250,1133,7812,675,106,16247,49540,641,1917,10098,44609,0,0,1,1,1,1,1,1,Normal,Normal,0
4,2019-12-15 23:30:12,172.24.1.80,38233,172.24.1.1,53,2,2,74,0,0,1,0,False,False,False,True,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,7441,31641,1868,25227,1491,30485,1448,22362,11583,40048,3208,173,4,6,1544,237,499,136,21195,25139,40,62,6578,33760,0,0,0,0,0,0,0,0,Normal,Normal,0


In [19]:
df.describe()


,Timestamp,Protocol,Service,Duration,Scr_bytes,Des_bytes,Conn_state,missed_bytes,Scr_pkts,Scr_ip_bytes,Des_pkts,Des_ip_bytes,anomaly_alert,total_bytes,total_packet,paket_rate,byte_rate,Scr_packts_ratio,Des_pkts_ratio,Scr_bytes_ratio,Des_bytes_ratio,Avg_user_time,Std_user_time,Avg_nice_time,Std_nice_time,Avg_system_time,Std_system_time,Avg_iowait_time,Std_iowait_time,Avg_ideal_time,Std_ideal_time,Avg_tps,Std_tps,Avg_rtps,Std_rtps,Avg_wtps,Std_wtps,Avg_ldavg_1,Std_ldavg_1,Avg_kbmemused,Std_kbmemused,Avg_num_Proc/s,Std_num_proc/s,Avg_num_cswch/s,std_num_cswch/s,OSSEC_alert,OSSEC_alert_level,Login_attempt,Succesful_login,File_activity,Process_activity,read_write_physical.process,is_privileged,class3
count,631814,631814.00000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.00000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000
mean,2020-01-26 04:25:42.960625152,1.48164,4.510323,59469.363612,5647.344600,9299.992150,0.852561,0.031535,247.740099,13939.486048,257.997646,12712.369716,0.097983,21959.722605,643.858705,101653.564798,97817.089666,12953.165193,1024.825358,36521.153591,5709.902595,6478.619622,25228.600552,1827.135333,27413.523931,1837.285239,29807.778626,1792.559149,9042.012798,8461.574751,25101.53133,1606.802323,9434.373034,557.261452,5203.388678,900.064291,4170.055287,937.533192,436.379555,15302.951528,24537.642184,123.325460,417.382503,7206.606345,23224.854796,0.051605,0.270216,0.082998,0.078504,0.068433,0.078396,0.356133,0.078233,0.485909
min,2019-05-09 09:39:16,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2020-01-08 22:54:09,1.00000,2.000000,94.000000,0.000000,0.000000,1.000000,0.000000,2.000000,1709.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,12868.000000,1.000000,0.000000,0.000000,5179.000000,12771.000000,1138.000000,15978.000000,1003.000000,17159.000000,57.000000,449.000000,2555.000000,11217.00000,773.000000,2314.000000,4.000000,6.000000,553.000000,317.000000,495.000000,83.000000,10120.000000,13051.000000,32.000000,31.000000,4411.000000,10064.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2020-01-21 05:28:00,1.00000,4.000000,4878.000000,2826.000000,1872.500000,1.000000,0.000000,3.000000,17056.000000,2.000000,8343.000000,0.000000,28267.000000,868.000000,104758.000000,85829.500000,16027.000000,280.000000,53534.000000,1521.000000,6251.000000,25589.000000,1644.000000,28972.000000,1425.000000,31530.000000,209.000000,4677.000000,9132.000000,25307.00000,1569.000000,8663.000000,287.000000,2245.000000,941.000000,3879.000000,762.000000,172.000000,15290.000000,24554.000000,59.000000,92.000000,7107.000000,21306.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2020-02-24 00:22:55,2.00000,4.000000,110575.750000,11375.000000,14400.000000,1.000000,0.000000,467.000000,24177.000000,634.000000,21504.000000,0.000000,33562.000000,868.000000,174147.000000,172180.750000,17145.000000,1159.000

In [20]:
_, var_dropped, corr_dropped = feature_analysis_pipeline(
    df=df, drop_columns=datesetInfo.drop_columns, label_col=datesetInfo.label_col)
var_dropped, corr_dropped

(['Bad_checksum', 'is_SYN_with_RST'],
 {'Avg_iowait_time',
  'Avg_num_Proc/s',
  'Avg_rtps',
  'Conn_state',
  'Des_pkts_ratio',
  'Duration',
  'File_activity',
  'Is_SYN_ACK',
  'Login_attempt',
  'OSSEC_alert',
  'Process_activity',
  'Succesful_login',
  'is_pure_ack',
  'is_syn_only'})

In [21]:
print(f"==>> type(var_dropped): {type(var_dropped)}")
print(f"==>> type(corr_dropped): {type(corr_dropped)}")


==>> type(var_dropped): <class 'list'>
==>> type(corr_dropped): <class 'set'>


In [22]:
var_dropped = set(var_dropped)
weak_columns = var_dropped.union(set(corr_dropped))
weak_columns


{'Avg_iowait_time',
 'Avg_num_Proc/s',
 'Avg_rtps',
 'Bad_checksum',
 'Conn_state',
 'Des_pkts_ratio',
 'Duration',
 'File_activity',
 'Is_SYN_ACK',
 'Login_attempt',
 'OSSEC_alert',
 'Process_activity',
 'Succesful_login',
 'is_SYN_with_RST',
 'is_pure_ack',
 'is_syn_only'}

### Attacks Types

In [23]:
df[datesetInfo.class_col].unique()


array(['Reconnaissance', 'Normal', 'RDOS', 'Weaponization',
       'Lateral _movement', 'Exfiltration', 'Exploitation', 'Tampering',
       'C&C', 'crypto-ransomware'], dtype=object)

In [24]:
# df[class_col] = df[class_col].replace({"BENIGN": "Benign",
#                                        "DDoS": "ddos",
#                                        "Web Attack � Brute Force": "bruteforce",
#                                        "Web Attack � XSS": "xss"})


In [25]:
classes = df[datesetInfo.class_col].unique()


### Sorting (optional)

In [26]:
if with_sort_timestamp and datesetInfo.timestamp_col:
    df[datesetInfo.timestamp_col] = pd.to_datetime(
        df[datesetInfo.timestamp_col].str.strip(), format=timestamp_format)
    df.sort_values(datesetInfo.timestamp_col, inplace=True)

### Encoding Attacks into integers

In [27]:
df, labels_names = one_dataset_class_num_col(
    df, datesetInfo.class_num_col, datesetInfo.class_col)

==>> labels_names: {np.int64(0): np.str_('C&C'), np.int64(1): np.str_('Exfiltration'), np.int64(2): np.str_('Exploitation'), np.int64(3): np.str_('Lateral _movement'), np.int64(4): np.str_('Normal'), np.int64(5): np.str_('RDOS'), np.int64(6): np.str_('Reconnaissance'), np.int64(7): np.str_('Tampering'), np.int64(8): np.str_('Weaponization'), np.int64(9): np.str_('crypto-ransomware')}


In [28]:
df.groupby(datesetInfo.class_col).size()


class2
C&C                    2216
Exfiltration          17202
Exploitation            887
Lateral _movement     23060
Normal               324810
RDOS                 110007
Reconnaissance        99118
Tampering              3940
Weaponization         50569
crypto-ransomware         5
dtype: int64

### Undersampling classes (optional)

In [29]:
if with_undersample_classes:
    # Group by the class column and get the count of records in each class
    class_counts = df.groupby(datesetInfo.class_col).size()

    # Sort the counts in descending order
    class_counts_sorted = class_counts.sort_values(ascending=False)
    print(f"==>> class_counts_sorted: {class_counts_sorted}")


In [30]:
if with_undersample_classes:
    # Get the classes with the highest number of records (you can choose how many to undersample)
    classes_to_undersample = class_counts_sorted.index[:2]

    # Undersample the classes with the highest number of records
    dfs = []
    for class_label in class_counts_sorted.index:
        print(f"==>> class_label: {class_label}")
        if class_label in classes_to_undersample:
            class_df = df[df[datesetInfo.class_col] == class_label]
            # Specify the fraction of samples to keep
            undersampled_df = class_df.sample(frac=0.5)
            dfs.append(undersampled_df)
        else:
            dfs.append(df[df[datesetInfo.class_col] == class_label])

    df = []
    # Optional: shuffle the undersampled DataFrame
    df = pd.concat(dfs).sample(frac=1).reset_index(drop=True)

In [31]:
if with_undersample_classes:
    # Group by the class column and get the count of records in each class
    class_counts = df.groupby(datesetInfo.class_col).size()

    # Sort the counts in descending order
    class_counts_sorted = class_counts.sort_values(ascending=False)
    print(f"==>> class_counts_sorted: {class_counts_sorted}")


### saving labels encodings and datasets properties

In [32]:
with open(folder_path + '/labels_names.pkl', 'wb') as f:
    pickle.dump([labels_names, classes], f)


In [33]:
df[datesetInfo.label_col].unique()


array([1, 0])

In [34]:
total_count = len(df)

properties = {
    "name": name,
    "length": total_count,
}

num_benign = len(df[df[datesetInfo.label_col] == 0])
num_attack = len(df[df[datesetInfo.label_col] == 1])

properties["num_benign"] = num_benign
properties["percentage_of_benign_records"] = ((num_benign * 100)/total_count)

properties["num_attack"] = num_attack
properties["percentage_of_attack_records"] = ((num_attack * 100)/total_count)

properties["attacks"] = list(df[datesetInfo.class_col].unique())

properties["weak_columns"] = list(weak_columns)

with open(folder_path + '/df_properties.json', 'w') as f:
    json.dump(properties, f)

properties

{'name': 'x_iiot',
 'length': 631814,
 'num_benign': 324810,
 'percentage_of_benign_records': 51.40911724020044,
 'num_attack': 307004,
 'percentage_of_attack_records': 48.59088275979956,
 'attacks': ['Reconnaissance',
  'Normal',
  'RDOS',
  'Weaponization',
  'Lateral _movement',
  'Exfiltration',
  'Exploitation',
  'Tampering',
  'C&C',
  'crypto-ransomware'],
 'weak_columns': ['Process_activity',
  'Login_attempt',
  'is_syn_only',
  'Avg_iowait_time',
  'Avg_num_Proc/s',
  'Duration',
  'Des_pkts_ratio',
  'is_SYN_with_RST',
  'Succesful_login',
  'OSSEC_alert',
  'is_pure_ack',
  'Conn_state',
  'Bad_checksum',
  'File_activity',
  'Avg_rtps',
  'Is_SYN_ACK']}

In [35]:
# graphs_properties_path = folder_path + '/graphs_properties.json'
# G = nx.from_pandas_edgelist(
#     df,
#     source=src_ip_col,
#     target=dst_ip_col,
#     create_using=nx.DiGraph()
# )
# calculate_graph_measures(G, graphs_properties_path)


In [36]:
df.to_parquet(output_path)


In [37]:
df.shape


(631814, 68)

In [38]:
df.head()


,Timestamp,Scr_IP,Scr_port,Des_IP,Des_port,Protocol,Service,Duration,Scr_bytes,Des_bytes,Conn_state,missed_bytes,is_syn_only,Is_SYN_ACK,is_pure_ack,is_with_payload,FIN or RST,Bad_checksum,is_SYN_with_RST,Scr_pkts,Scr_ip_bytes,Des_pkts,Des_ip_bytes,anomaly_alert,total_bytes,total_packet,paket_rate,byte_rate,Scr_packts_ratio,Des_pkts_ratio,Scr_bytes_ratio,Des_bytes_ratio,Avg_user_time,Std_user_time,Avg_nice_time,Std_nice_time,Avg_system_time,Std_system_time,Avg_iowait_time,Std_iowait_time,Avg_ideal_time,Std_ideal_time,Avg_tps,Std_tps,Avg_rtps,Std_rtps,Avg_wtps,Std_wtps,Avg_ldavg_1,Std_ldavg_1,Avg_kbmemused,Std_kbmemused,Avg_num_Proc/s,Std_num_proc/s,Avg_num_cswch/s,std_num_cswch/s,OSSEC_alert,OSSEC_alert_level,Login_attempt,Succesful_login,File_activity,Process_activity,read_write_physical.process,is_privileged,class1,class2,class3,Class
0,2020-01-09 03:35:56,192.168.2.199,49278,192.168.2.10,80,1,4,75756,1849,13843,1,0,True,True,True,True,True,False,False,25,7529,57,17208,1,917,913,167694,43725,16027,1159,44278,13909,11423,27335,1455,27902,4139,42866,113,1057,2193,40997,333,734,1640,494,1026,5953,2047,1005,16196,27966,574,1178,13398,3974,1,5,0,0,0,0,0,0,Scanning_vulnerability,Reconnaissance,1,6
1,2020-01-12 23:31:13,10.0.1.5,39769,131.236.3.92,53,2,2,83,16696,1,1,0,False,False,False,True,False,False,False,296,3276,1,1,0,29023,868,148000,100503,17185,1,58186,1,2744,37376,2400,35644,3253,45533,4108,23469,3505,44749,2892,17266,1,1,1206,7535,578,190,6190,41653,78,238,8311,16722,0,0,0,0,0,0,0,0,Normal,Normal,0,4
2,2020-01-08 22:28:06,172.24.1.80,59050,172.24.1.1,53,2,2,132,10806,14586,1,0,False,False,False,True,False,False,False,2,25357,2,32292,0,28664,868,98902,41993,16027,1159,53534,4653,9190,35462,223,1981,660,6866,3804,21387,11589,33713,2438,16476,1126,12046,1269,6490,513,62,25823,21218,78,2,8932,25165,0,0,0,0,0,0,0,0,Normal,Normal,0,4
3,2020-02-26 22:54:00,192.168.2.196,37966,192.168.2.10,1880,1,16,260329,689,16960,1,0,True,True,True,True,True,False,False,1020,4938,1013,39562,0,38772,443,62894,148816,16906,280,56990,1197,1256,44269,1406,33028,3414,39860,249,4971,6960,2339,2040,12882,926,9250,1133,7812,675,106,16247,49540,641,1917,10098,44609,0,0,1,1,1,1,1,1,Normal,Normal,0,4
4,2019-12-15 23:30:12,172.24.1.80,38233,172.24.1.1,53,2,2,74,0,0,1,0,False,False,False,True,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,7441,31641,1868,25227,1491,30485,1448,22362,11583,40048,3208,173,4,6,1544,237,499,136,21195,25139,40,62,6578,33760,0,0,0,0,0,0,0,0,Normal,Normal,0,4


In [39]:
df.describe(include="all")


,Timestamp,Scr_IP,Scr_port,Des_IP,Des_port,Protocol,Service,Duration,Scr_bytes,Des_bytes,Conn_state,missed_bytes,is_syn_only,Is_SYN_ACK,is_pure_ack,is_with_payload,FIN or RST,Bad_checksum,is_SYN_with_RST,Scr_pkts,Scr_ip_bytes,Des_pkts,Des_ip_bytes,anomaly_alert,total_bytes,total_packet,paket_rate,byte_rate,Scr_packts_ratio,Des_pkts_ratio,Scr_bytes_ratio,Des_bytes_ratio,Avg_user_time,Std_user_time,Avg_nice_time,Std_nice_time,Avg_system_time,Std_system_time,Avg_iowait_time,Std_iowait_time,Avg_ideal_time,Std_ideal_time,Avg_tps,Std_tps,Avg_rtps,Std_rtps,Avg_wtps,Std_wtps,Avg_ldavg_1,Std_ldavg_1,Avg_kbmemused,Std_kbmemused,Avg_num_Proc/s,Std_num_proc/s,Avg_num_cswch/s,std_num_cswch/s,OSSEC_alert,OSSEC_alert_level,Login_attempt,Succesful_login,File_activity,Process_activity,read_write_physical.process,is_privileged,class1,class2,class3,Class
count,631814,631814,631814,631814,631814,631814.00000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814,631814,631814,631814,631814,631814,631814,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.00000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814.000000,631814,631814,631814.000000,631814.000000
unique,NaN,33,54702,94,6333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2,2,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,10,NaN,NaN
top,NaN,192.168.2.199,5353,192.168.2.10,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,True,True,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,Normal,NaN,NaN
freq,NaN,121087,2226,241863,181771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,321834,384518,384300,548488,318077,631814,631814,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,324810,324810,NaN,NaN
mean,2020-01-26 04:25:42.960625152,NaN,NaN,NaN,NaN,1.48164,4.510323,59469.363612,5647.344600,9299.992150,0.852561,0.031535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,247.740099,13939.486048,257.997646,12712.369716,0.097983,21959.722605,643.858705,101653.564798,97817.089666,12953.165193,1024.825358,36521.153591,5709.902595,6478.619622,25228.600552,1827.135333,27413.523931,1837.285239,29807.778626,1792.559149,9042.012798,8461.574751,25101.53133,1606.802323,9434.373034,557.261452,5203.388678,900.064291,4170.055287,937.533192,436.379555,15302.951528,24537.642184,123.325460,417.382503,7206.606345,23224.854796,0.051605,0.270216,0.082998,0.078504,0.068433,0.078396,0.356133,0.078233,NaN,NaN,0.485909,4.691754
min,2019-05-09 09:39:16,NaN,NaN,NaN,NaN,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000
25%,2020-01-08 22:54:09,NaN,NaN,NaN,NaN,1.00000,2.000000,94.000000,0.000000,0.000000,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,1709.0000

In [40]:
df.dtypes


Timestamp        datetime64[ns]
Scr_IP                   object
Scr_port                 object
Des_IP                   object
Des_port                 object
                      ...      
is_privileged             int64
class1                   object
class2                   object
class3                    int64
Class                     int64
Length: 68, dtype: object

In [41]:
list(df.dtypes)


[dtype('<M8[ns]'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('bool'),
 dtype('bool'),
 dtype('bool'),
 dtype('bool'),
 dtype('bool'),
 dtype('bool'),
 dtype('bool'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 

In [42]:
for column, dtype in df.dtypes.items():
    print(f"Column '{column}': {dtype}")


Column 'Timestamp': datetime64[ns]
Column 'Scr_IP': object
Column 'Scr_port': object
Column 'Des_IP': object
Column 'Des_port': object
Column 'Protocol': int64
Column 'Service': int64
Column 'Duration': int64
Column 'Scr_bytes': int64
Column 'Des_bytes': int64
Column 'Conn_state': int64
Column 'missed_bytes': int64
Column 'is_syn_only': bool
Column 'Is_SYN_ACK': bool
Column 'is_pure_ack': bool
Column 'is_with_payload': bool
Column 'FIN or RST': bool
Column 'Bad_checksum': bool
Column 'is_SYN_with_RST': bool
Column 'Scr_pkts': int64
Column 'Scr_ip_bytes': int64
Column 'Des_pkts': int64
Column 'Des_ip_bytes': int64
Column 'anomaly_alert': int64
Column 'total_bytes': int64
Column 'total_packet': int64
Column 'paket_rate': int64
Column 'byte_rate': int64
Column 'Scr_packts_ratio': int64
Column 'Des_pkts_ratio': int64
Column 'Scr_bytes_ratio': int64
Column 'Des_bytes_ratio': int64
Column 'Avg_user_time': int64
Column 'Std_user_time': int64
Column 'Avg_nice_time': int64
Column 'Std_nice_time